In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# for Saving the model
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# !unzip '/content/drive/My Drive/TEST_DATASET.zip' -d '/content/drive/My Drive'
# !unzip '/content/drive/My Drive/test_data_2.zip' -d '/content/drive/My Drive'


# /content/drive/My Drive/TRAIN_DATASET.zip

In [ ]:
TRAIN_DIR = "/content/drive/My Drive/TRAIN_DATASET"
TEST_DIR = "/content/drive/My Drive/TEST_DATASET" 

In [ ]:
import numpy as np
import pandas as pd 
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
from keras.models import Sequential
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation,GlobalMaxPooling2D
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.applications import VGG16
from keras.models import Model


In [ ]:
image_size = 400
input_shape = (image_size, image_size, 3)

# epochs = 10
batch_size = 32

pre_trained_model = VGG16(input_shape=input_shape, include_top=False, weights="imagenet")
    
for layer in pre_trained_model.layers[:15]:
    layer.trainable = False

for layer in pre_trained_model.layers[15:]:
    layer.trainable = True
    
last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output
    
# Flatten the output layer to 1 dimension
x = GlobalMaxPooling2D()(last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = Dense(512, activation='relu')(x)
# Add a dropout rate of 0.5
x = Dropout(0.5)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(3, activation='softmax')(x)

model = Model(pre_trained_model.input, x)
model.summary()

In [ ]:
# model.load_weights('/content/gdrive/My Drive/MyCNN/epochs:047-val_acc:0.905.hdf5') /content/drive/My Drive/checkpoints #epochs:022-accuracy:0.859.hdf5

model.load_weights('/content/drive/My Drive/checkpoints/epochs:022-accuracy:0.859.hdf5')  #/content/drive/My Drive/checkpoints #epochs:022-accuracy:0.859.hdf5

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

In [ ]:
import os
filenames = os.listdir("/content/drive/My Drive/TRAIN_DATASET")
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'Transmission':
        categories.append(1)
    elif category == 'trans_and_dist':
        categories.append(2)
    else:
        categories.append(0)

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})
df

In [ ]:
df[5000:5010]

In [ ]:
df['category'].value_counts().plot.bar()

In [ ]:
train_df, validate_df = train_test_split(df, test_size=0.20, random_state=42)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [ ]:
total_train = train_df.shape[0]
total_validate = validate_df.shape[0]
batch_size=15

total_train,total_validate

In [ ]:
TRAIN_SIZE = len([name for name in os.listdir(TRAIN_DIR)])
TEST_SIZE = len([name for name in os.listdir(TEST_DIR)])
print("Number of training images:", TRAIN_SIZE)
print("Number of test images:", TEST_SIZE)

VALID_FRACTION = 0.2
BATCH_SIZE = 32
# EPOCHS = 30
import tensorflow as tf
IMAGE_WIDTH = IMAGE_HEIGHT = 250

# creating df with train labels
train_filenames = os.listdir(TRAIN_DIR)
train_labels = []
for filename in train_filenames:
    label = filename.split('.')[0]
    train_labels.append(label)

train_df = pd.DataFrame({
    'id': train_filenames,
    'label': train_labels
})


# splitting to train & valid
train_df, valid_df = train_test_split(train_df, test_size=VALID_FRACTION)

# augmentation settings, for now just normalizing
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(    
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255.,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
    )

# not doing any data augmentation on validation test set
valid_datagen  = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)

# creating train and valid generators (not using valid_split to avoid doing data augmentation on validation set)
train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    TRAIN_DIR, 
    x_col='id',
    y_col='label',
    target_size=(image_size, image_size),
    class_mode='categorical',
    batch_size=BATCH_SIZE
)

valid_generator = valid_datagen.flow_from_dataframe(
    valid_df, 
    TRAIN_DIR, 
    x_col='id',
    y_col='label',
    target_size=(image_size, image_size),
    class_mode='categorical',
    batch_size=BATCH_SIZE
)

In [ ]:
train_df

In [ ]:
# train_df, validate_df = train_test_split(df, test_size=0.20, random_state=42)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [ ]:
from keras.callbacks import *
filepath="/content/drive/My Drive/checkpoints/epochs:{epoch:03d}-accuracy:{accuracy:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
EPOCHS = 35
# training
history = model.fit(train_generator,
    validation_data=valid_generator,
    steps_per_epoch= round(TRAIN_SIZE*(1.-VALID_FRACTION)/BATCH_SIZE),
    validation_steps= round(TRAIN_SIZE*VALID_FRACTION/BATCH_SIZE),
    epochs= EPOCHS, 
    initial_epoch = 23,
    callbacks=[callbacks_list],
    verbose=1)

In [ ]:
model.save('colab_VGG16_TD2.h5')
model_file = drive.CreateFile({'Vision_1' : 'colab_VGG16_TD2.h5'})
model_file.SetContentFile('colab_VGG16_TD2.h5')
model_file.Upload()

In [ ]:
import os

IMAGE_WIDTH = IMAGE_HEIGHT = 250
BATCH_SIZE = 32

TEST_DIR = "/content/drive/My Drive/TEST_DATASET"
TEST_SIZE = len([name for name in os.listdir(TEST_DIR)])



test_filenames = os.listdir(TEST_DIR)
test_df = pd.DataFrame({
    'id': test_filenames
})

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0/255.)

test_generator = test_datagen.flow_from_dataframe(
    test_df, 
    TEST_DIR, 
    x_col='id',
    y_col=None,
    class_mode=None,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=False
)


In [ ]:
predict = model.predict_generator(test_generator, steps=np.ceil(TEST_SIZE/BATCH_SIZE))
test_df['category'] = np.argmax(predict, axis=-1)
label_map = dict((v,k) for k,v in train_generator.class_indices.items())
test_df['category'] = test_df['category'].replace(label_map)
test_df['category'].value_counts().plot.bar()

In [ ]:
#3 Writing the output
from keras.preprocessing.image import ImageDataGenerator, load_img
import cv2
import matplotlib.pyplot as plt
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 2
thickness = 2
IMAGE_SIZE = (IMAGE_WIDTH, IMAGE_HEIGHT)
sample_test = test_df.head(22)
sample_test.head()
plt.figure(figsize=(50, 50)) #12, 24)
for index, row in sample_test.iterrows():
    filename = row['id']
    category = row['category']
    img = cv2.imread("/content/drive/My Drive/TEST_DATASET/"+filename)
    img = cv2.resize(img, (480,640))
    cv2.putText(img, category,(30, 50),font, fontScale, (0,255,0),thickness,True)
    output_file_path = '/content/drive/My Drive/output2/' + filename
    cv2.imwrite(output_file_path,img)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, load_img
import matplotlib.pyplot as plt

IMAGE_SIZE = (IMAGE_WIDTH, IMAGE_HEIGHT)

sample_test = test_df.head(18)
sample_test.head()
plt.figure(figsize=(30, 30)) #12, 24)
for index, row in sample_test.iterrows():
    filename = row['id']
    category = row['category']
    img = load_img("/content/drive/My Drive/TEST_DATASET/"+filename, target_size=IMAGE_SIZE)
    plt.subplot(6, 3, index+1)
    plt.imshow(img)
    plt.xlabel(filename + '(' + "{}".format(category) + ')' )
plt.tight_layout()
plt.show()